## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

In [84]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize as wt
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from sklearn.pipeline import make_pipeline
import numpy as np
import pickle

In [46]:
data = pd.read_csv('women-clothing-accessories.3-class.balanced.csv', sep='\t')

In [47]:
data.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [48]:
data.describe()

,review,sentiment
count,90000,90000
unique,87321,3
top,Товар не пришёл,negative
freq,58,30000


Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [49]:
nltk.download('stopwords')
nltk.download('punkt');

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
stop_words = stopwords.words('russian')
punctuation = string.punctuation

In [51]:
from IPython.display import clear_output

lemmatizer = MorphAnalyzer()

def preprocessing(reviews, *, verbose=True):
    i = 0
    res = []
    for review in reviews:
        if verbose:
            clear_output(wait=True)
            print(f'{i}/{len(reviews)}, {(i / len(reviews) * 100):.2f}%')
            i += 1
            
        review = review.lower()
        for punct in punctuation:
            review = review.replace(punct, ' ')
        review = wt(review)
        review = list(map(lambda x: lemmatizer.parse(x)[0].normal_form, review))
        res.append(' '.join(review))
    return res

In [52]:
X = list(data.review)
X = preprocessing(X)

89999/90000, 100.00%


In [53]:
y = data.sentiment

In [54]:
# save preprocessed data 

with open("preprocessed data", "wb") as fp:
    pickle.dump(X, fp)

In [55]:
# load preprocessed data

with open("preprocessed data", "rb") as fp:
    X = pickle.load(fp)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [57]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [58]:
def ngram_range(min_n, max_n_from, max_n_to):
    for i in range(min_n, max_n_from):
        for j in range(i, max_n_to):
            yield (i, j)

In [59]:
import os
if os.name == 'nt':
    proc = !echo %NUMBER_OF_PROCESSORS%
    proc = proc[0]
else:
    proc = !nproc

free_cores = 8
proc = int(proc) - free_cores
proc

12

#### CountVectorizer words

In [62]:
from sklearn.feature_extraction.text import CountVectorizer

In [63]:
model = make_pipeline(CountVectorizer(stop_words=stop_words, analyzer='word'),
                      MultinomialNB())

param_grid = {
    'countvectorizer__ngram_range' : list(ngram_range(1, 5, 5))}

search = GridSearchCV(model, refit=True, verbose=3, n_jobs=proc, scoring='f1_micro', param_grid=param_grid)

In [64]:
search.fit(X_train, y_train)
search.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'countvectorizer__ngram_range': (1, 2)}

In [65]:
pred = search.predict(X_test)
report = classification_report(y_test, pred)
words_rep = classification_report(y_test, pred, output_dict=True)
print(report)

              precision    recall  f1-score   support

    neautral       0.61      0.63      0.62     10026
    negative       0.71      0.66      0.68      9819
    positive       0.82      0.86      0.84      9855

    accuracy                           0.71     29700
   macro avg       0.71      0.71      0.71     29700
weighted avg       0.71      0.71      0.71     29700



#### TfidfVectorizer

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [67]:
model = make_pipeline(TfidfVectorizer(stop_words=stop_words, analyzer='word'),
                      MultinomialNB())

param_grid = {
    'tfidfvectorizer__ngram_range' : list(ngram_range(1, 5, 5)),
    'tfidfvectorizer__min_df' : np.linspace(0, 0.5, 4),
    'tfidfvectorizer__max_df' : np.linspace(0.5, 1, 4)}
    'tfidfvectorizer__max_features' '

search = GridSearchCV(model, refit=True, verbose=3, n_jobs=proc, scoring='f1_micro', param_grid=param_grid)

In [68]:
search.fit(X_train, y_train)
search.best_params_

Fitting 5 folds for each of 160 candidates, totalling 800 fits


C:\Users\Bill\anaconda3\envs\pytorch\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
520 fits failed out of a total of 800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
520 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Bill\anaconda3\envs\pytorch\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Bill\anaconda3\envs\pytorch\lib\site-packages\sklearn\pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\Bill\anaconda3\envs\pytorch\lib\site-packages\sklearn\pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_t

{'tfidfvectorizer__max_df': 0.5,
 'tfidfvectorizer__min_df': 0.0,
 'tfidfvectorizer__ngram_range': (1, 2)}

In [69]:
pred = search.predict(X_test)
report = classification_report(y_test, pred)
tfidf_rep = classification_report(y_test, pred, output_dict=True)
print(report)

              precision    recall  f1-score   support

    neautral       0.61      0.62      0.62     10026
    negative       0.71      0.66      0.68      9819
    positive       0.82      0.86      0.84      9855

    accuracy                           0.71     29700
   macro avg       0.71      0.71      0.71     29700
weighted avg       0.71      0.71      0.71     29700



#### CountVectorizer char

In [70]:
model = make_pipeline(CountVectorizer(stop_words=stop_words, analyzer='char'),
                      MultinomialNB())

param_grid = {
    'countvectorizer__ngram_range' : list(ngram_range(1, 10, 10))}

search = GridSearchCV(model, refit=True, verbose=3, n_jobs=proc, scoring='f1_micro', param_grid=param_grid)

In [71]:
search.fit(X_train, y_train)
search.best_params_

Fitting 5 folds for each of 45 candidates, totalling 225 fits


{'countvectorizer__ngram_range': (7, 9)}

In [72]:
pred = search.predict(X_test)
report = classification_report(y_test, pred)
char_rep = classification_report(y_test, pred, output_dict=True)
print(report)

              precision    recall  f1-score   support

    neautral       0.62      0.68      0.64     10026
    negative       0.71      0.66      0.68      9819
    positive       0.87      0.85      0.86      9855

    accuracy                           0.73     29700
   macro avg       0.73      0.73      0.73     29700
weighted avg       0.73      0.73      0.73     29700



#### Report

In [88]:
reportdf = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1'])
for report in [words_rep, tfidf_rep, char_rep]:
    reportdf.loc[len(reportdf.index)] = [report['accuracy'], report['weighted avg']['precision'], report['weighted avg']['recall'], report['weighted avg']['f1-score']]

reportdf['name'] = ['CountVectorizer words', 'TfidfVectorizer', 'CountVectorizer char']

reportdf.insert(0, 'name', reportdf.pop('name'))
reportdf.sort_values('f1', ascending=False)

,name,accuracy,precision,recall,f1
2,CountVectorizer char,0.727374,0.732188,0.727374,0.728896
0,CountVectorizer words,0.714512,0.714356,0.714512,0.713850
1,TfidfVectorizer,0.712862,0.711455,0.712862,0.711574


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [27]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [89]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [90]:
text = data.review[0]
text

'качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......'

In [91]:
re.findall(r"\b\w{2}", text)

['ка',
 'пл',
 'по',
 'уж',
 'го',
 'на',
 'Фо',
 'не',
 'со',
 'Тк',
 'уж',
 'ри',
 'бл',
 'ма',
 'ру',
 'не',
 'та',
 'УЖ',
 'не',
 'ст',
 'за',
 'та',
 'де']

### split
разделяет строку по заданному шаблону


In [92]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [93]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [94]:
text = 'Какие-то странные предложения. Зачем. Ставить. Столько. Точек. Никто не знает'
text

'Какие-то странные предложения. Зачем. Ставить. Столько. Точек. Никто не знает'

In [95]:
re.split(r"\.", text, maxsplit=2)

['Какие-то странные предложения',
 ' Зачем',
 ' Ставить. Столько. Точек. Никто не знает']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [96]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [97]:
text = 'Хотелось бы скрыть номер карты в этом предложении 4485 5647 8756 7909'
text

'Хотелось бы скрыть номер карты в этом предложении 4485 5647 8756 7909'

In [37]:
re.sub(r'\d', 'DIG', text)

'Хотелось бы скрыть номер карты в этом предложении DIGDIGDIGDIG DIGDIGDIGDIG DIGDIGDIGDIG DIGDIGDIGDIG'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [98]:
text = 'И зачем здесь ссылка на википедию https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D0%B3%D0%BC%D0%B0_(%D0%B0%D0%BB%D0%B3%D0%B5%D0%B1%D1%80%D0%B0) ?'
text

'И зачем здесь ссылка на википедию https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D0%B3%D0%BC%D0%B0_(%D0%B0%D0%BB%D0%B3%D0%B5%D0%B1%D1%80%D0%B0) ?'

In [99]:
re.sub(r'http[s]:\/\/[\w./%\d(){}]*', '', text)

'И зачем здесь ссылка на википедию  ?'

### compile
компилирует регулярное выражение в отдельный объект

In [100]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [101]:
text = 'Магма (группоид) в общей алгебре — алгебра, состоящая из множества М с одной бинарной операцией M × M → M. Помимо требования замкнутости множества относительно заданной на нём операции, других требований к операции и множеству не предъявляется.'
text

'Магма (группоид) в общей алгебре — алгебра, состоящая из множества М с одной бинарной операцией M × M → M. Помимо требования замкнутости множества относительно заданной на нём операции, других требований к операции и множеству не предъявляется.'

In [102]:
regexp = re.compile(r'\w{4,}')
regexp.findall(text)

['Магма',
 'группоид',
 'общей',
 'алгебре',
 'алгебра',
 'состоящая',
 'множества',
 'одной',
 'бинарной',
 'операцией',
 'Помимо',
 'требования',
 'замкнутости',
 'множества',
 'относительно',
 'заданной',
 'операции',
 'других',
 'требований',
 'операции',
 'множеству',
 'предъявляется']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [103]:
text = 'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz'
text

'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz'

In [104]:
regexp = re.compile(r'@\w*.\w*')
regexp.findall(text)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']